# Database Setup

**Goal:** Get semantic search up and running using Postgres and `pgvector`

## Getting Started with RDS

If you're just looking to experiment with vector search/RAG storing your embeddings in a local database will do just fine. If, however, you'd like to call on this database in another application, I'd be in your interest to host your database using a service like AWS. 

Getting started on AWS is easy: 
- Navigate to the RDS service page on the AWS Console
- Create a new RDS Postgres instance, making sure that your instance is publically accessible
- Review your VPC settings and ensure your inbound/outbound rules allow for the appropriate connections

That's a high level overview, but it should get you 90% of the way there

## Creating a Vector Enabled Database in Postgres

This project will use a free-tier Postgres RDS instance on AWS to store embedded content for our RAG system. Postgres offers a native vector extension that will allow us to work with semantic search in a familiar database driver. Read more about it on the [`pgvector` repo page](https://github.com/pgvector/pgvector)

After running the following commands, you're ready to start upserting embedded content into your vector database!

In [ ]:
-- create a database for our rag system 
CREATE DATABASE resume_rag;

In [ ]:
-- enable the vector extension 
CREATE EXTENSION vector;

-- enable a simple uuid extension
CREATE EXTENSION IF NOT EXISTS "uuid-ossp";

In [ ]:
-- create a new table to store our vector embeddings
CREATE TABLE content_embeddings (
    uid UUID PRIMARY KEY DEFAULT uuid_generate_v4(), 
    document_id INTEGER,
    chunk_id INTEGER,
    tags TEXT,
    clean_text TEXT,
    embedding VECTOR (384)
);

In [ ]:
-- add a tsvector column for lexical search 
ALTER TABLE content_embeddings
ADD COLUMN search_terms tsvector
GENERATED ALWAYS AS (
    to_tsvector('english', 
        COALESCE(clean_text, '') || ' ' || COALESCE(tags, ' ')
    )
) STORED;

In [ ]:
-- create an index on our tsvector column 
CREATE INDEX lexicalsearch_index ON content_embeddings USING GIN (search_terms);